In [ ]:
model_id = "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"
hf_token = ""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Guarda tu token de Hugging Face en un archivo en tu Google Drive
with open("/content/drive/MyDrive/hf_token.txt", "w") as file:
    file.write("")

# Lee el token desde el archivo y autentica
with open("/content/drive/MyDrive/hf_token.txt", "r") as file:
    hf_token = file.read().strip()

# Iniciar sesión en Hugging Face usando tu token
!huggingface-cli login --token $hf_token


Mounted at /content/drive
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `embeddings_asl` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `embeddings_asl`


In [ ]:
pip install -U sentence-transformers

In [ ]:
pip install tensorflow tensorflow-hub scikit-learn pandas


In [ ]:
plats = "/content/terminos_contexto_generado_inst_1shot_limpio.csv"
jap = "/content/eurovoc_contexto_generado_llama8b_inst_3shot_limpio.csv"
gold ="/content/gold_standard_enlazado.csv"c

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score


# Cargar los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)
gold_standard = pd.read_csv(gold)

# Limpiar nombres de columnas
plats_df.columns = plats_df.columns.str.strip()
jap_df.columns = jap_df.columns.str.strip()
gold_standard.columns = gold_standard.columns.str.strip()

# Cargar el modelo de embeddings
model_id = "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"
model = SentenceTransformer(model_id)

# Crear las oraciones combinando Término + contexto
plats_df['sentence'] = plats_df['Termino'].astype(str) + "," + plats_df['Ventana_de_contexto'].astype(str)
jap_df['sentence'] = jap_df['Término'].astype(str) + "," + jap_df['Ventana_de_contexto'].astype(str)

# Generar embeddings
plats_embeddings = model.encode(plats_df['sentence'].tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_df['sentence'].tolist(), convert_to_tensor=True)

# Normalizar los embeddings para similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular matriz de similitud
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Umbral de similitud
umbral = 0.8

# Buscar matches con similitud mayor al umbral
detected_matches = []
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_term = plats_df['Termino'].iloc[i]
            jap_term = jap_df['Término'].iloc[j]
            detected_matches.append((plats_term, jap_term, similarity))

# Evaluar contra gold-standard
gold_pairs = set(zip(gold_standard['Termino_1'], gold_standard['Término_2']))
detected_pairs = set((plats_term, jap_term) for plats_term, jap_term, _ in detected_matches)

# y_true / y_pred para métricas
y_true = [1 if pair in gold_pairs else 0 for pair in detected_pairs]
y_pred = [1] * len(detected_pairs)

# Añadir los positivos del gold que no fueron detectados
for pair in gold_pairs:
    if pair not in detected_pairs:
        y_true.append(1)
        y_pred.append(0)

# Calcular métricas
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir matches
for plats_term, jap_term, similarity in detected_matches:
    print(f"Plats: {plats_term} - Jap: {jap_term} - Similitud: {similarity:.4f}")


Detected Matches: 26
Precision: 0.5769
Recall: 0.2381
F1 Score: 0.3371
Plats: trabajo nocturno - Jap: trabajo nocturno - Similitud: 0.8997
Plats: maternidad - Jap: maternidad - Similitud: 0.8017
Plats: remuneración - Jap: remuneración del trabajo - Similitud: 0.8527
Plats: remuneración - Jap: salario - Similitud: 0.8275
Plats: contrato - Jap: contrato - Similitud: 0.8395
Plats: contrato - Jap: contrato de trabajo - Similitud: 0.8824
Plats: mesa electoral - Jap: mesa electoral - Similitud: 0.8441
Plats: riesgo - Jap: riesgo cubierto - Similitud: 0.9150
Plats: jornada - Jap: jornada de trabajo - Similitud: 0.8446
Plats: votación - Jap: votación - Similitud: 0.8596
Plats: contrato de trabajo - Jap: contrato de trabajo - Similitud: 0.9154
Plats: trabajador con discapacidad - Jap: trabajador con discapacidad - Similitud: 0.8134
Plats: consanguinidad - Jap: parentesco - Similitud: 0.8238
Plats: empresario - Jap: empresario - Similitud: 0.8661
Plats: salud pública - Jap: salud pública - Simil